In [1]:
import pandas as pd # library for data analsysis

! python -m pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

! python -m pip install folium
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np # library to handle data in a vectorized manner

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

     |████████████████████████████████| 93 kB 2.6 MB/s  eta 0:00:01
Libraries imported.


Alabama Cities and their Latitudes and Longitudes were retrieved from https://simplemaps.com/data/us-cities

In [2]:
!wget -q -O 'al_cities_lat_long.csv' https://www.dropbox.com/s/r823yayzncnqe9q/al_cities_lat_long.csv?dl=0
print('Data downloaded!')

Data downloaded!


Now that the data is downloaded, let's read it into a _pandas_ dataframe and display the number of cities in our data set.

In [3]:
al_cities = pd.read_csv('al_cities_lat_long.csv')
print(al_cities.shape)
al_cities.head()

(200, 3)


,City,Latitude,Longitude
0,Birmingham,33.5277,-86.7987
1,Huntsville,34.6988,-86.6412
2,Mobile,30.6783,-88.1162
3,Montgomery,32.3473,-86.2666
4,Tuscaloosa,33.2348,-87.5267


In [4]:
address = 'Alabama'

geolocator = Nominatim(user_agent="alabama")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Alabama are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Alabama are 33.2588817, -86.8295337.


In [5]:
# create map of Alabama using latitude and longitude values
map_al = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, label in zip(al_cities['Latitude'], al_cities['Longitude'], al_cities['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_al)  
    
map_al

Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'ACBYMCUUSNDYT423KHRGMWJMWIBGLPI004MZXBSQC1CSFC0X' # your Foursquare ID
CLIENT_SECRET = 'RIFHLYBKVY5WIIP44AFTSMWCSHNYURVEOITPCU5JTM0LSPRJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ACBYMCUUSNDYT423KHRGMWJMWIBGLPI004MZXBSQC1CSFC0X
CLIENT_SECRET:RIFHLYBKVY5WIIP44AFTSMWCSHNYURVEOITPCU5JTM0LSPRJ


Let's create a function to obtain all venues for the cities in Alabama.

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's run the above function on each city in Alabama and create a new dataframe called al_venues.

In [8]:
al_venues = getNearbyVenues(names=al_cities['City'],
                                   latitudes=al_cities['Latitude'],
                                   longitudes=al_cities['Longitude']
                                  )

Birmingham
Huntsville
Mobile
Montgomery
Tuscaloosa
Auburn
Hoover
Florence
Anniston
Dothan
Daphne
Decatur
Gadsden
Madison
Enterprise
Albertville
Foley
Phenix City
Prattville
Vestavia Hills
Alabaster
Athens
Opelika
Bessemer
Northport
Homewood
Pelham
Fairhope
Trussville
Prichard
Oxford
Mountain Brook
Valley
Helena
Cullman
Selma
Tillmans Corner
Sylacauga
Troy
Center Point
Millbrook
Hueytown
Calera
Saraland
Muscle Shoals
Hartselle
Gardendale
Chelsea
Talladega
Jasper
Moody
Irondale
Gulf Shores
Jacksonville
Pell City
Leeds
Ozark
Saks
Fairfield
Pike Road
Clay
Scottsboro
Robertsdale
Russellville
Boaz
Rainbow City
Pleasant Grove
Meadowbrook
Fultondale
Forestdale
Spanish Fort
Alexander City
Southside
Sheffield
Bay Minette
Guntersville
Wetumpka
Eufaula
Tuscumbia
Arab
Tuskegee
Brook Highland
Pinson
Fort Payne
Moores Mill
Lincoln
Montevallo
Brewton
Meridianville
Hamilton
Theodore
Andalusia
Orange Beach
Satsuma
Lanett
Tarrant
Grayson Valley
Clanton
Atmore
Attalla
Chickasaw
Demopolis
Harvest
Semmes
Hi

Let's check the size of the resulting dataframe.

In [78]:
print(al_venues.shape)
al_venues

(590, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Birmingham,33.5277,-86.7987,The Pit Barbeque,33.530252,-86.800837,BBQ Joint
1,Birmingham,33.5277,-86.7987,Sarris,33.526543,-86.794331,American Restaurant
2,Birmingham,33.5277,-86.7987,Subway,33.526873,-86.801567,Sandwich Place
3,Birmingham,33.5277,-86.7987,Birm/Jeff Bus Barn,33.528248,-86.794528,Bus Station
4,Huntsville,34.6988,-86.6412,Redstone Pool,34.695808,-86.643733,Pool
...,...,...,...,...,...,...,...
585,Bridgeport,34.9495,-85.7243,MeMaw's,34.949116,-85.721802,American Restaurant
586,Greensboro,32.7014,-87.5950,PieLab,32.704189,-87.595226,Café
587,Greensboro,32.7014,-87.5950,CVS pharmacy,32.700775,-87.594643,Pharmacy
588,Greensboro,32.7014,-87.5950,Dollar General,32.705495,-87.595302,Discount Store


Let's check how many venues were returned for each city.

In [10]:
al_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Abbeville,7,7,7,7,7,7
Alabaster,7,7,7,7,7,7
Albertville,4,4,4,4,4,4
Alexander City,1,1,1,1,1,1
Alexandria,1,1,1,1,1,1
...,...,...,...,...,...,...
Tuscumbia,2,2,2,2,2,2
Union Springs,2,2,2,2,2,2
Vestavia Hills,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [11]:
print('There are {} uniques categories.'.format(len(al_venues['Venue Category'].unique())))

There are 149 uniques categories.


##  Analyze Each City in Alabama

In [111]:
# one hot encoding
al_onehot = pd.get_dummies(al_venues[['Venue Category']], prefix="", prefix_sep="")

# add City column back to dataframe
al_onehot['City'] = al_venues['City'] 

# move city column to the first column
fixed_columns = [al_onehot.columns[-1]] + list(al_onehot.columns[:-1])
al_onehot = al_onehot[fixed_columns]

al_onehot.head()

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Women's Store
0,Birmingham,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Birmingham,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Birmingham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Birmingham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Huntsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [13]:
al_onehot.shape

(590, 150)

#### Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [14]:
al_grouped = al_onehot.groupby('City').mean().reset_index()
al_grouped

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Women's Store
0,Abbeville,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
1,Alabaster,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
2,Albertville,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
3,Alexander City,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
4,Alexandria,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Tuscumbia,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
119,Union Springs,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0
120,Vestavia Hills,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000,0.0,0.25,0.0,0.0,0.0,0.0
121,Wetumpka,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.125,0.0,0.00,0.0,0.0,0.0,0.0


#### Let's confirm the new size

In [15]:
al_grouped.shape

(123, 150)

#### Let's print each city along with the top 10 most common venues

In [84]:
num_top_venues = 10

for hood in al_grouped['City']:
    print("----"+hood+"----")
    temp = al_grouped[al_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbeville----
                           venue  freq
0                    Gas Station  0.29
1                          Diner  0.14
2               Business Service  0.14
3                           Bank  0.14
4                 Discount Store  0.14
5            American Restaurant  0.14
6                          Plaza  0.00
7  Paper / Office Supplies Store  0.00
8                    Music Venue  0.00
9                     Nail Salon  0.00


----Alabaster----
                           venue  freq
0             Chinese Restaurant  0.14
1                    Pizza Place  0.14
2              Convenience Store  0.14
3                       Pharmacy  0.14
4           Fast Food Restaurant  0.14
5                            Gym  0.14
6                    Gas Station  0.14
7  Paper / Office Supplies Store  0.00
8                    Music Venue  0.00
9                     Nail Salon  0.00


----Albertville----
                        venue  freq
0               Grocery Store  0.25
1         

#### Let's put that into a _pandas_ dataframe

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

After writing the above function, let's create the new dataframe and display the top 10 venues for each city.

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = al_grouped['City']

for ind in np.arange(al_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(al_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbeville,Gas Station,American Restaurant,Diner,Discount Store,Business Service,Bank,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,Alabaster,Pizza Place,Pharmacy,Gas Station,Chinese Restaurant,Fast Food Restaurant,Gym,Convenience Store,Dessert Shop,Diner,Department Store
2,Albertville,Construction & Landscaping,Grocery Store,Discount Store,Performing Arts Venue,Women's Store,Drugstore,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
3,Alexander City,BBQ Joint,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Women's Store
4,Alexandria,Insurance Office,Women's Store,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
...,...,...,...,...,...,...,...,...,...,...,...
118,Tuscumbia,Construction & Landscaping,Insurance Office,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
119,Union Springs,Fried Chicken Joint,Food,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
120,Vestavia Hills,Food,Travel & Transport,Scenic Lookout,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore
121,Wetumpka,American Restaurant,Construction & Landscaping,Train Station,Sporting Goods Shop,Gastropub,Discount Store,Food,Diner,Dessert Shop,Doctor's Office


## Clustering Cities

Run _k_-means to cluster the cities into 9 clusters.

In [87]:
# set number of clusters
kclusters = 9

al_grouped_clustering = al_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(al_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([5, 0, 5, 5, 5, 5, 0, 5, 5, 0, 5, 0, 0, 7, 5, 5, 5, 5, 0, 5],
      dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each city.

In [113]:
# add clustering labels

city_venues_sorted = city_venues_sorted.drop(columns=['Cluster Labels'])

city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

al_merged = al_cities


# merge manhattan_grouped with manhattan_data to add latitude/longitude for each city
al_merged = al_merged.join(city_venues_sorted.set_index('City'), on='City')

al_merged = al_merged.dropna() #drop NaN rows without Cluster Label values

al_merged = al_merged.reset_index(drop=True) #Reset index

al_merged = al_merged.astype({"Cluster Labels":'int'}) #Convert 'Cluster Label' column from Float to Int so the map works

print(al_merged.shape)
al_merged.head() # check the last columns!

(123, 14)


,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Birmingham,33.5277,-86.7987,5,American Restaurant,Bus Station,Sandwich Place,BBQ Joint,Women's Store,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,Huntsville,34.6988,-86.6412,5,Pool,Food,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore,Donut Shop
2,Mobile,30.6783,-88.1162,5,Breakfast Spot,Intersection,Fast Food Restaurant,Flower Shop,Brazilian Restaurant,Chinese Restaurant,Jewelry Store,Bank,Japanese Restaurant,Automotive Shop
3,Montgomery,32.3473,-86.2666,5,Steakhouse,Gym,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
4,Tuscaloosa,33.2348,-87.5267,5,Food & Drink Shop,Food,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore,Donut Shop


In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(al_merged['Latitude'], al_merged['Longitude'], al_merged['City'], al_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [103]:
al_merged.loc[al_merged['Cluster Labels'] == 0, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] #fast food/pizza popular

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Auburn,Fast Food Restaurant,Deli / Bodega,Discount Store,Liquor Store,Sandwich Place,Mediterranean Restaurant,Mexican Restaurant,Coffee Shop,Doctor's Office,Dog Run
17,Alabaster,Pizza Place,Pharmacy,Gas Station,Chinese Restaurant,Fast Food Restaurant,Gym,Convenience Store,Dessert Shop,Diner,Department Store
31,Muscle Shoals,Fast Food Restaurant,Video Store,Convenience Store,Mexican Restaurant,Medical Center,Women's Store,Dry Cleaner,Fish & Chips Shop,Farmers Market,Farm
32,Hartselle,Pizza Place,Discount Store,Fast Food Restaurant,Video Store,Food,Pharmacy,Asian Restaurant,Gas Station,Supermarket,Women's Store
34,Talladega,Fast Food Restaurant,Mexican Restaurant,Pharmacy,Liquor Store,Café,Pizza Place,Video Store,Diner,Discount Store,Doctor's Office
35,Jasper,Fast Food Restaurant,Pizza Place,Pharmacy,Department Store,Shoe Store,Supplement Shop,Chinese Restaurant,Mobile Phone Shop,Shopping Mall,Food
37,Pell City,Salon / Barbershop,Burger Joint,Ice Cream Shop,Fast Food Restaurant,Women's Store,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore
46,Boaz,Fast Food Restaurant,Gas Station,Deli / Bodega,Gymnastics Gym,Kids Store,Pharmacy,Pizza Place,Gym,Discount Store,Clothing Store
57,Arab,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Women's Store,Drugstore,Fish & Chips Shop,Farmers Market,Farm,Electronics Store,Dry Cleaner
60,Brewton,Pizza Place,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop


#### Cluster 2

In [104]:
al_merged.loc[al_merged['Cluster Labels'] == 1, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] #parks most popular

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Foley,Park,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
59,Montevallo,Park,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
99,Livingston,Bookstore,Park,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
115,Taylor,Hardware Store,Park,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner


#### Cluster 3

In [105]:
al_merged.loc[al_merged['Cluster Labels'] == 2, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] #construction and landscaping popular

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Dothan,Construction & Landscaping,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
10,Decatur,Construction & Landscaping,Fast Food Restaurant,Women's Store,Drugstore,Fish & Chips Shop,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
23,Oxford,Construction & Landscaping,Discount Store,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
44,Robertsdale,Construction & Landscaping,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
56,Tuscumbia,Construction & Landscaping,Insurance Office,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
73,Margaret,Construction & Landscaping,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
82,Priceville,Mexican Restaurant,Construction & Landscaping,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner


#### Cluster 4

In [107]:
al_merged.loc[al_merged['Cluster Labels'] == 3, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] ## baseball fields, women's stores

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Sylacauga,Baseball Field,Women's Store,Dry Cleaner,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Drugstore
85,Holtville,Baseball Field,Women's Store,Dry Cleaner,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Drugstore
96,Heflin,Gym / Fitness Center,Baseball Field,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
118,Good Hope,Intersection,Baseball Field,Women's Store,Dry Cleaner,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store


#### Cluster 5

In [108]:
al_merged.loc[al_merged['Cluster Labels'] == 4, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] #discount stores, drugstores, women's stores

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Forestdale,Grocery Store,Discount Store,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
62,Theodore,Discount Store,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
76,Glencoe,Discount Store,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
84,Holt,Fish & Chips Shop,Discount Store,Women's Store,Drugstore,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
109,Moundville,Discount Store,Video Store,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner


#### Cluster 6

In [98]:
al_merged.loc[al_merged['Cluster Labels'] == 5, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] #large city mix

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Birmingham,American Restaurant,Bus Station,Sandwich Place,BBQ Joint,Women's Store,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,Huntsville,Pool,Food,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore,Donut Shop
2,Mobile,Breakfast Spot,Intersection,Fast Food Restaurant,Flower Shop,Brazilian Restaurant,Chinese Restaurant,Jewelry Store,Bank,Japanese Restaurant,Automotive Shop
3,Montgomery,Steakhouse,Gym,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
4,Tuscaloosa,Food & Drink Shop,Food,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...
112,Abbeville,Gas Station,American Restaurant,Diner,Discount Store,Business Service,Bank,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
113,Geneva,Grocery Store,Moving Target,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
116,Carlisle-Rockledge,Flower Shop,Italian Restaurant,Furniture / Home Store,Garden Center,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
117,Henagar,Mexican Restaurant,Sandwich Place,Discount Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore


#### Cluster 7

In [99]:
al_merged.loc[al_merged['Cluster Labels'] == 6, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]#golf course/vacation spots

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Gulf Shores,Golf Course,Women's Store,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore
65,Grayson Valley,Golf Course,Candy Store,Women's Store,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner


#### Cluster 8

In [100]:
al_merged.loc[al_merged['Cluster Labels'] == 7, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] # american restaurants

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
114,Point Clear,American Restaurant,Women's Store,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
119,Midland City,American Restaurant,Women's Store,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
121,Bridgeport,American Restaurant,Women's Store,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner


#### Cluster 9

In [109]:
al_merged.loc[al_merged['Cluster Labels'] == 8, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]] #home service

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Madison,Home Service,Cave,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Drugstore
47,Rainbow City,Home Service,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Women's Store
49,Meadowbrook,Home Service,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Women's Store
86,Mount Olive,Home Service,Drugstore,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Women's Store
97,Piedmont,Home Service,Café,Drugstore,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop


#### Cluster 10

In [102]:
al_merged.loc[al_merged['Cluster Labels'] == 9, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 11

In [150]:
al_merged.loc[al_merged['Cluster Labels'] == 10, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 12

In [151]:
al_merged.loc[al_merged['Cluster Labels'] == 11, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 13

In [152]:
al_merged.loc[al_merged['Cluster Labels'] == 12, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 14

In [153]:
al_merged.loc[al_merged['Cluster Labels'] == 13, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 15

In [156]:
al_merged.loc[al_merged['Cluster Labels'] == 14, al_merged.columns[[0] + list(range(4, al_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [179]:
al_merged.loc[al_merged['City'] == 'Huntsville']

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
137,Huntsville,34.6988,-86.6412,1,Pool,Women's Store,Dog Run,Fishing Store,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
